In [4]:
import tweepy
from pymongo import MongoClient
import json
import pandas 
import unicodedata
from subprocess import check_output
#from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [5]:
client = MongoClient('localhost', 27017)
db = client.newtweetsDB
db.cooltweets.count()

443

In [6]:
cursor = db.cooltweets.find() #Where tweets is my collection
tweet_fields = ['text','id']
docs = pandas.DataFrame(list(cursor), columns = tweet_fields)

In [7]:
doc_complete  = docs['text'].tolist()


In [32]:
import gensim
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]    

In [33]:
import re
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
doc_cleanNew = []
for doc in doc_clean:
    newdoc =  []
    for word in doc:
        #print(word)
        word  =  emoji_pattern.sub(r'', word)
        word =    word.encode('ascii',errors='ignore')
        if (word != ''):
            newdoc.append(word)
        #print(word)
    #print(doc)
    #print(newdoc)
    doc_cleanNew.append(newdoc)
        
#print(doc_cleanNew)
        


In [34]:
corpora.Dictionary(doc_cleanNew)
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
dictionary = corpora.Dictionary(doc_cleanNew)

In [35]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_cleanNew]

In [36]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=4 , id2word = dictionary, passes=100)
print(ldamodel.print_topics(num_topics= 4, num_words= 6))

[(0, u'0.006*one + 0.005*rohan + 0.005*ji + 0.004*amp + 0.004*b + 0.004*support'), (1, u'0.019*gurmeetramrahim + 0.016*effectoflionheart + 0.007*westworldfinaleonswp + 0.007*swpremiere + 0.006*it + 0.005*haryana'), (2, u'0.005*ki + 0.004*ji + 0.004*like + 0.004*r + 0.004*via + 0.004*kha'), (3, u'0.014*here + 0.013*sipsip + 0.013*httpstcom7u54y9l6y + 0.013*song + 0.013*raviraj + 0.013*mistaabaaz')]


In [37]:
stri = ' '
for wrd in doc_cleanNew:
    
    for indword in wrd:
        if (indword != 'rt'):
            stri = stri + indword + ' '

In [38]:
stri

' ive always enjoyed traveling experience different culture different httpstcoukikvrd6xf cc whowilltweeet httpstco325bvuwjxe mgurudath hain aise bos sabko mile gt 145 lakh cr estd nincompoop govt gifted 2 lac cr funding fake currency holder httpstcoflt0l29bjl swpremiere a8 c theresa charlotte westworldfinaleonswp biggboss yes rohan go support rohan rbganesh1 httpstcom7u54y9l6y please share song charan raviraj mistaabaaz sipsip here aaryababbar222 delhi gov working overnight change face govt school one govt school delhi httpstcoikt49wqclc narendramodi httpstcoanbc8is93j gurmeetramrahim efectoflionheart goodnight pita ji happy manveer plying good absence chutia mannu bb10 emotweet httpstcotj5ujiyeon meghanavirat madbwoyy yo wassup simple logic bigger currency noteshigher risk hoardingampcorruptionampviceversa smaller onesif so lie wisdomjsb iamsrk anupamachopra love cnni legible stay ever dc httpstcom7u54y9l6y please share song charan raviraj mistaabaaz sipsip here isoniamann anupampkher

In [39]:
wordcloud = WordCloud( stopwords=STOPWORDS,   background_color='white',   
                      width=1200,   height=1000     ).generate(stri)

plt.imshow(wordcloud)
plt.axis('off')
plt.show()